In [ ]:
# option 1: use correlation between numerical vars to remove

corr_matrix = num_features.corr()

# Extract only highly correlated columns
threshold = 0.8
highly_correlated_cols = corr_matrix[abs(corr_matrix) > threshold].stack().reset_index()
highly_correlated_cols = highly_correlated_cols[
    highly_correlated_cols["level_0"] != highly_correlated_cols["level_1"]
]
highly_correlated_cols = highly_correlated_cols.rename(columns={0: "correlation"})
highly_correlated_cols["abs_correlation"] = highly_correlated_cols["correlation"].abs()
highly_correlated_cols = highly_correlated_cols.sort_values(
    "abs_correlation", ascending=False
)
highly_correlated_cols = highly_correlated_cols.drop("abs_correlation", axis=1)

# Print highly correlated columns with correlation coefficients
for idx, row in highly_correlated_cols.iterrows():
    col1 = row["level_0"]
    col2 = row["level_1"]
    corr = row["correlation"]
    print(f"- {col1} & {col2}: {corr}")

In [ ]:
features.drop(['GarageCars', 
                  'GarageYrBlt',
                  'TotRmsAbvGrd'], 
                  axis = 1, inplace = True)

In [ ]:
# option 2: remove highly correlated cols (> .9) after get_dummies

dummies = pd.get_dummies(features)
dummies.shape

# can also specify columns to dummify
#churn = pd.get_dummies(churn, columns=col_2_dummy, drop_first=True)

In [ ]:
corr_matrix = dummies.corr()

# Extract only highly correlated columns
threshold = 0.9
highly_correlated_cols = corr_matrix[abs(corr_matrix) > threshold].stack().reset_index()
highly_correlated_cols = highly_correlated_cols[
    highly_correlated_cols["level_0"] != highly_correlated_cols["level_1"]
]
highly_correlated_cols = highly_correlated_cols.rename(columns={0: "correlation"})
highly_correlated_cols["abs_correlation"] = highly_correlated_cols["correlation"].abs()
highly_correlated_cols = highly_correlated_cols.sort_values(
    "abs_correlation", ascending=False
)
highly_correlated_cols = highly_correlated_cols.drop("abs_correlation", axis=1)

# Print highly correlated columns with correlation coefficients
for idx, row in highly_correlated_cols.iterrows():
    col1 = row["level_0"]
    col2 = row["level_1"]
    corr = row["correlation"]
    print(f"- {col1} & {col2}: {corr}")

In [ ]:
# Columns to remove:  

# CentralAir_N
# Utilities_NoSewr
# Exterior2nd_PreCast
# BsmtFinType1_None and BsmtQual_None and BsmtCond_None and BsmtFinType2_None
# Street_Grvl
# Exterior2nd_PreCast
# GarageFinish_None and GarageCond_None and GarageType_None
# SaleCondition_Partial
# Exterior2nd_VinylSd and Exterior2nd_CmentBd and Exterior2nd_MetalSd
# LandSlope_Mod
# RoofStyle_Hip
# LotShape_IR1
# PoolQC_None
# ExterQual_TA

In [ ]:
dummies.drop(['CentralAir_N', 'Utilities_NoSewr', 
              'Exterior2nd_PreCast', 'BsmtFinType1_None', 'BsmtQual_None',
              'BsmtCond_None', 'BsmtFinType2_None', 'Street_Grvl', 'Exterior2nd_PreCast',
              'GarageFinish_None', 'GarageCond_None', 'GarageType_None', 
              'SaleCondition_Partial', 'Exterior2nd_VinylSd', 'Exterior2nd_CmentBd', 'Exterior2nd_MetalSd', 
              'LandSlope_Mod', 'RoofStyle_Hip', 'LotShape_IR1', 
              'PoolQC_None', 'ExterQual_TA'], axis = 1, inplace = True)

In [ ]:
corr_matrix = dummies.corr()

# Extract only highly correlated columns
threshold = 0.9
highly_correlated_cols = corr_matrix[abs(corr_matrix) > threshold].stack().reset_index()
highly_correlated_cols = highly_correlated_cols[
    highly_correlated_cols["level_0"] != highly_correlated_cols["level_1"]
]
highly_correlated_cols = highly_correlated_cols.rename(columns={0: "correlation"})
highly_correlated_cols["abs_correlation"] = highly_correlated_cols["correlation"].abs()
highly_correlated_cols = highly_correlated_cols.sort_values(
    "abs_correlation", ascending=False
)
highly_correlated_cols = highly_correlated_cols.drop("abs_correlation", axis=1)

# Print highly correlated columns with correlation coefficients
for idx, row in highly_correlated_cols.iterrows():
    col1 = row["level_0"]
    col2 = row["level_1"]
    corr = row["correlation"]
    print(f"- {col1} & {col2}: {corr}")
    
# returns nothing

In [ ]:
# option 3: use Label Encoder, then vif

# values are so high, this is not useful

from sklearn.preprocessing import LabelEncoder
cat_features_encoded = cat_features.apply(LabelEncoder().fit_transform)
cat_features_encoded.head()

In [ ]:
features_encoded = pd.concat([num_features, cat_features_encoded], axis = 1)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
vif_data = pd.DataFrame()
vif_data["feature"] = features_encoded.columns
vif_data["VIF"] = [variance_inflation_factor(features_encoded.values, i)
                          for i in range(len(features_encoded.columns))]
vif_data.sort_values(by = 'VIF', ascending = False)

In [ ]:
# option 4: use f regression selection

# with p-value < 0.5, reject the null hypothesis that there is no relationship

In [ ]:
from sklearn.feature_selection import f_regression

res = f_regression(features_encoded[:], housing['SalePrice'])
corr_tab = pd.DataFrame({
    'feature': features_encoded.columns,
    'f_regression': res[0],
    'p-value': res[1]
})

corr_tab.sort_values(by = 'p-value', ascending=True)

In [ ]:
corr_vars = corr_tab[corr_tab['p-value'] <= 0.05]
corr_vars.shape # 68 vars (11 removed)
features_fregression = corr_vars['feature']
features_fregression